### Librerías

In [127]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt; import seaborn as sns

### Lectura y preprocesamiento de datos

In [125]:
df = pd.read_csv('mission_launches.csv').drop(columns=['Unnamed: 0.1','Unnamed: 0'])

df.rename(columns={'Company Name':'company',
                   'Location':'launch_facility',
                   'Datum':'date',
                   'Detail':'detail',
                   'Status Rocket':'rocket_status',
                   ' Rocket':'cost',
                   'Status Mission':'mission_status'},
          inplace=True)


#Cantidad de valores nulos y filas repetidas

# print(f'Hay un total de {df.isna().sum().sum()} valores nulos distribuidos así:',
#       f'\nCost: {df["cost"].isna().sum()} ({df["cost"].isna().sum()/df.shape[0]*100:.2f}% del total)')

##Conversión de la columna de fechas. Algunos valores de la columna tiene fecha y hora pero otros solo tienen fecha.
df['date'] = df['date'].str.replace(' UTC','')

def conversor_fechas(fechas):
    if ':' in fechas:
        return pd.to_datetime(fechas, format= '%a %b %d, %Y %H:%M')
    else:
        return pd.to_datetime(fechas, format= '%a %b %d, %Y')
df['date'] = df['date'].apply(conversor_fechas)

df['cost'] = df['cost'].str.replace(',','')
df['cost'] = df['cost'].astype(float)

##Creación de medidas derivadas

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

##Obtener el país de lanzamiento del cohete
df['launch_location'] = df['launch_facility'].str.split(', ').str[-1]

##La columna detail tiene el nombre del cohete y la carga que lleva, hay que dividirla

df[['rocket','load']] = df['detail'].str.split('|',expand=True)
df.drop(columns='detail',inplace=True)

##Es necesario reasignar algunos lugares de lanzamiento que no dicen directamente el nombre del país.
df['launch_location'].value_counts();

dict_changes = {
    'New Mexico' : 'USA',
    "Yellow Sea": 'China',
    "Shahrud Missile Test Site": "Iran",
    "Pacific Missile Range Facility": 'USA',
    "Barents Sea": 'Russia',
    "Gran Canaria": 'USA'
}
df['launch_location'] = df['launch_location'].replace(dict_changes)

##Organizar los datos por fecha

df = df.sort_values('date').reset_index(drop=True)